## Content
- SimpleRenderer
    - Usage
    - Sizing
- RemoteRenderer
    - Usage
    - Using Jupyter Server Proxy
    - Using a custom protocol
    - Usage with a pvserver and choosen port¶

# SimpleRenderer

Does nothing more than show the ParaViewWeb server application in the Output Area.

## Usage

Start a ParaViewWeb server application over the command line (see the [ParaView example](https://kitware.github.io/paraviewweb/examples/RemoteRenderer.html#Using-ParaView-as-server))
```bash
pvpython pv_server.py --port 1234 --authKey wslink-secret 
```

In [1]:
from pvlink import SimpleRenderer

simple = SimpleRenderer(sessionURL='ws://localhost:1234/ws', authKey='wslink-secret')
display(simple)

SimpleRenderer(sessionURL='ws://localhost:1234/ws')

## Sizing
Widgets scales with container size.

In [2]:
from ipywidgets import Box

Box(children=[simple], layout={'height':'500px'})

Box(children=(SimpleRenderer(sessionURL='ws://localhost:1234/ws'),), layout=Layout(height='500px'))

# RemoteRenderer

This renderer requires the `paraview.simple` and `paraview.web` modules.

Upon initialization, the RemoteRenderer starts a webserver for you. If nothing is specified, the webserver will try to start on port 8080 or the next free port thereafter and create a random authentication key. You can pass your own arguments to the webserver. To display help on the possible arguments, you can call `<yourRenderer>.webserver_arguments_help()`.

If `pvserver_host` and `pvserver_port` (default 11111) are specified, the webserver will try to establish a connection to the ParaView server (pvserver) at the given host and port. The pvserver can then take over the heavy lifting and handle very large geometries. Locally, only the image data of the processed data is recieved. To prevent data from being rendered locally, we recommend using the `SetRecommendedRenderSettings` function from `pvlink.utilities` on your displayed view.

## Usage 
### Default

In [3]:
from pvlink import RemoteRenderer

renderer = RemoteRenderer(port=8082)
# Alternatively, if you want to render your data with a ParaView server:
# Start a pvserver over the command line and run the following line
# renderer = RemoteRenderer(pvserver_host='localhost', pvserver_port=11111, p=8082)
display(renderer)

RemoteRenderer(authKey='1412e786544799a40e7231cd3e06d7bc22ecb0d6d16b978c', sessionURL='ws://localhost:8082/ws'…

At this point, there is no view or sources, so the output will be a blank canvas.

In [4]:
from paraview import simple
from pvlink.utility import SetRecommendedRenderSettings

# Create a view and...
view = simple.CreateView('RenderView', 'example')
# ...disable interactor-based render calls and
# ensure pvserver-side rendering (if applicable)
SetRecommendedRenderSettings(view)
# Create and show a source
source = simple.Cone()
simple.Show(source, view)
# Update the renderer widget to display the changes
renderer.update_render()

### Using Jupyter Server Proxy

If you want to access your webserver using Jupyter Server Proxy, you need to set `use_jupyter_server_proxy` to True and specify the baseURL. Setting `use_jupyter_server_proxy_https` to true, allows to use the https certificates of the proxy, to use an encrypted connection between client brwoser and the jupyter Server Proxy.

Example: If your notebook url is `http://localhost:8888` and you would access a process using `http://localhost:88888/proxy/8080`, the baseURL would be the part before 'proxy, `localhost:8888`.

In [5]:
proxied_renderer = RemoteRenderer(baseURL='localhost:8888', use_jupyter_server_proxy=True, 
                                  use_jupyter_server_proxy_https=True, port=8080, ws='pvwebserver/ws')

If no `viewID` is specified, the widget will always show the active view. To bind it to a view, we need the GlobalID of the view and pass it to the renderer.

In [6]:
proxied_renderer.viewID = view.GetGlobalIDAsString()
display(proxied_renderer)

RemoteRenderer(authKey='2d94f211be34b2c58ea069de4182f23f5d3986b9732ebed0', sessionURL='ws://localhost:8888/pro…

You can display help for the possible webserver arugments.

In [7]:
proxied_renderer.webserver_arguments_help()

ParaView Web Server

optional arguments:
  -h, --help            show this help message and exit
  -d, --debug           log debugging messages to stdout
  -s, --nosignalhandlers
                        Prevent Twisted to install the signal handlers so it
                        can be started inside a thread.
  -i HOST, --host HOST  the interface for the web-server to listen on
                        (default: localhost)
  -p PORT, --port PORT  port number for the web-server to listen on (default:
                        8080)
  -t TIMEOUT, --timeout TIMEOUT
                        timeout for reaping process on idle in seconds
                        (default: 300s)
  -c CONTENT, --content CONTENT
                        root for web-pages to serve (default: none)
  -a AUTHKEY, --authKey AUTHKEY
                        Authentication key for clients to connect to the
                        WebSocket.
  -f, --force-flush     If provided, this option will force additional padding
   

## Using a custom protocol
You can define your own pipeline which should be run when the webserver starts up.

First, define a class which inherits from `pv_wslink.PVServerProtocol` and add your pipeline there. Per default, the RemoteRenderer shows the active view.

To ensure that the view you created in the pipeline is the one shown in the rendering widget and does not get replaced by a new active view, we need to make it available to the outside and later on bind the viewID to the rendering widget. 

In [8]:
from paraview import simple
from paraview.web import pv_wslink
from paraview.web import protocols as pv_protocols

from pvlink.utility import SetRecommendedRenderSettings


class DemoServer(pv_wslink.PVServerProtocol):
    authKey = 'wslink-secret'

    def initialize(self):
        # Bring used components
        self.registerVtkWebProtocol(pv_protocols.ParaViewWebMouseHandler())
        self.registerVtkWebProtocol(pv_protocols.ParaViewWebViewPort())
        self.registerVtkWebProtocol(pv_protocols.ParaViewWebViewPortImageDelivery())
        # Update authentication key to use
        self.updateSecret(DemoServer.authKey)

        # Your pipeline
        demo_view = simple.CreateView('RenderView', 'SphereView')
        SetRecommendedRenderSettings(demo_view)
        demo_viewID = demo_view.GetGlobalIDAsString()
        # Make the viewID available so we can bind our widget to the correct view
        self.setSharedObject('viewID', demo_viewID)
        sphere = simple.Sphere()
        simple.Show(sphere, demo_view)

In [9]:
custom_renderer = RemoteRenderer(protocol=DemoServer, a='mysecretkey')
# Bind the viewID to the widget to avoid it showing a different view
# when a different view is set to the active view
custom_renderer.viewID = custom_renderer.protocol.getSharedObject('viewID')
display(custom_renderer)

RemoteRenderer(authKey='mysecretkey', sessionURL='ws://localhost:8081/ws', viewID='555')

Since we did not bind a view to `renderer`, the sphere should now also be visible in the `renderer` [output](#Default) after an update call.

In [10]:
renderer.update_render()

## Usage with a pvserver and choosen port
This shows, how to connect to a pvserver on startup and open a specific port. Additionaly debug outpout is enabled, in case an error is searched. The pvserver adress and port can be specified. This only works with the default protocol, as the connection is established there. So in case of a custom protocol, the connection to the pvserver has to be started by the user (or in the custom protocol).

In [ ]:
renderer = RemoteRenderer(pvserver_host="localhost", pvserver_port = 11111, baseURL="localhost", ws="pvwebserver/ws",
                          debug=True, port = 1234)